In [3]:
from utils import *
from modules import *
import pandas as pd


In [4]:
import pickle

with open('logs/exp2023-07-28T06:22:40.937579/metadata.pkl', 'rb') as f:
    data = pickle.load(f)
data

{'args': Namespace(batch_size=50, cuda=True, decoder='mlp', decoder_dropout=0.0, decoder_hidden=128, dims=1, dynamic_graph=False, edge_types=2, encoder='mlp', encoder_dropout=0.0, encoder_hidden=128, epochs=100, factor=True, gamma=0.5, hard=False, load_folder='', lr=0.0005, lr_decay=200, no_cuda=False, no_factor=False, num_atoms=52, prediction_steps=10, prior=False, save_folder='logs', seed=42, skip_first=False, suffix='_springs5', temp=0.5, timesteps=30, var=5e-05)}

In [5]:
encoder = MLPEncoder(30, 128, 2, 0.0)

Using factor graph MLP encoder.


/home/jovyan/NRI-master/modules.py:27: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data)
/home/jovyan/NRI-master/modules.py:110: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  nn.init.xavier_normal(m.weight.data)


In [6]:
import os
encoder_file = os.path.join('logs/exp2023-07-28T06:22:40.937579', 'encoder.pt')
encoder.load_state_dict(torch.load(encoder_file))

<All keys matched successfully>

In [7]:
num_atoms = 52
off_diag = np.ones([num_atoms, num_atoms]) - np.eye(num_atoms)

rel_rec = np.array(encode_onehot(np.where(off_diag)[0]), dtype=np.float32)
rel_send = np.array(encode_onehot(np.where(off_diag)[1]), dtype=np.float32)
rel_rec = torch.FloatTensor(rel_rec)
rel_send = torch.FloatTensor(rel_send)

In [8]:
train_loader, valid_loader, test_loader, loc_max, loc_min, vel_max, vel_min = my_load_data(
    batch_size=50)

    Мы получили количество графов такое, какое количество батчей. Теперь мы хотим усреднить это дело перед округлением и получить граф из усредненных вероятностей. Причем не в виде тензора. Почти все придется переписать.

In [10]:
def np_str_not_diag_to_graph(str_not_diag, dim, accuracy = 0.5):

    coef = accuracy
    str_not_diag[(str_not_diag > coef) | (str_not_diag < -coef)] = 1
    str_not_diag[str_not_diag < 1] = 0
    
    str_not_diag = np.round(str_not_diag)
    k = 0
    mas = [[0]*dim for i in range(dim)]
    
    for i in range(dim):
        for j in range(dim):
            if i != j:
                mas[i][j] = str_not_diag[k]
                k += 1
            else:
                mas[i][j] = 1 #Диагональ
    mas = np.array(mas)
    return mas

In [11]:
preobr_edgeses = []
k = 0
for batch_idx, (data, relations) in enumerate(train_loader):

    temp = 0.5
    hard = False
    k += 1

    logits = encoder(data, rel_rec, rel_send)
    edges = gumbel_softmax(logits, tau=temp, hard=hard)
    prob = my_softmax(logits, -1)
    
    edges = torch.transpose(edges.mean(dim=0), 0, 1)
    str_not_diag = edges[1]
    str_not_diag = str_not_diag.detach().numpy()
    preobr_edgeses.append(str_not_diag)
preobr_edgeses = np.array(preobr_edgeses).mean(axis=0)
preobr_edgeses

array([0.4920662 , 0.4866867 , 0.43010366, ..., 0.68355286, 0.66998726,
       0.637368  ], dtype=float32)

In [27]:
copy_preobr_edgeses = preobr_edgeses.copy()
my_matrix = np_str_not_diag_to_graph(copy_preobr_edgeses, 52, 0.6)
my_matrix.sum()

580.0

In [23]:
pd.DataFrame(my_matrix).to_csv('/home/jovyan/NRI_dima.csv', sep = '\t', index = False, header=False)